# Doing Things with Text 3b: Import and clean one or more csv files

This notebook introduces the automatic cleaning and saving of one or more csv files that contains text. 

### Step 1: Setting Up NLTK

NLTK (Natural Language Toolkit) is a library for working with text. To use it, you'll need to download some additional language data the first time you use NLTK. Run the following cell once:

In [ ]:
# Import NLTK and download required packages
import nltk
nltk.download('punkt')  # Tokenizer
nltk.download('stopwords')  # Stopwords

### Step 2: Importing Required Packages

Here, we're loading a few packages to help with text cleaning:
- `os`: Helps with interacting with the operating system, such as managing file paths and directories.
- `csv`
- `glob`
- `re`: For regular expressions (patterns used for finding and cleaning text).
- `tqdm.notebook`
- `nltk.tokenize`: For splitting text into individual words.
- `nltk.corpus.stopwords`: A collection of common words like 'the', 'and', 'is', which are often removed in analysis.
-  `pandas`: Provides tools for handling and analyzing structured data in tables, making it easier to work with datasets.
-  `matplotlib.pyplot`: Allows for creating visualizations like charts and graphs to represent data visually.

In [ ]:
import os
import csv
import glob
import re
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize  # needs to be installed first via nltk.download()
from nltk.corpus import stopwords  # needs to be installed first via nltk.download()
import pandas as pd
import matplotlib.pyplot as plt

### Step 3: Define Input and Output Paths

Define where your text file is located (input) and where you want to save your processed text (output). You will use `os.path.join()` to define your paths. This approach is cross-platform, meaning it will work on Windows, macOS, and Linux.

Replace 'path', 'to', 'your', 'input', 'folder' with the actual paths to your files. It is not necessary for the output folder to exist. If it doesn't, this code will create it for you.

In [ ]:
# Define input and output paths
indir = '/Users/huijn001/desktop/test/'
outdir = '/Users/huijn001/desktop/test1/'
os.makedirs(os.path.dirname(outdir), exist_ok=True) # makes outdir if it doesn't exist already

allfiles = glob.glob(os.path.join(indir, "*.csv"))

dataset = 'dataset' # here the name of your actual dataset for output files

check what's in 'allfiles':

In [ ]:
for file in allfiles:
    print(file)

#### Step 3.1 check what the data structure of csv's looks like (change 'file.csv' for one of the actual files in indir)

In [ ]:
df_test = pd.read_csv(indir + '1977.csv', sep='\t') # most common separators are ';' or ',' or '\t'
print(df_test.head())

In [ ]:
cols_to_keep = ['year', 'text']
index_col = 'year' # preferably, the date column

### Step 4: Import csv's as df (with df['text']) as the text column), merge into one large dataframe called 'data'

'Data' will have at least four columns:
* the raw text from the input csv (usually ['text'])
* the cleaned text as a list (['text_clean'])
* the cleaned text as a string (['text_clean_str'])
* the word count based on text_clean (['word_count'])

#### Option 1: importing csv's with raw text, preprocessing including tokenization (takes time)

Default is one or more csv files with the filename ending in a year (for example text_1978.csv, 1851.csv, etc.). Make sure that the separator (sep=) is correctly defined (same as above) and that the text column has the correct header (from cols_to_keep)

In [ ]:
data = []  # Use a list to collect DataFrames

for file in tqdm(allfiles):
    # Load the file
    df = pd.read_csv(file, sep="\t", usecols=cols_to_keep, index_col=index_col)
    df['text'] = df['text'].str.lower()  # Lowercase text

    # Original word count per row
    df['original_word_count'] = df['text'].str.split().str.len()
    
    # Clean text
    df['text_clean'] = [
        [w for w in word_tokenize(text) if w.isalpha() and len(w) >= 4] 
        for text in df['text']
    ]
    
    # Cleaned word count per row
    df['cleaned_word_count'] = df['text_clean'].str.len()
    
    # Words removed per row
    df['words_removed'] = df['original_word_count'] - df['cleaned_word_count']
    
    # Convert cleaned text back to a string for further processing if needed
    df['text_clean_str'] = df['text_clean'].apply(' '.join)

    # Append the processed DataFrame
    data.append(df)

# Concatenate all DataFrames at once
data = pd.concat(data, axis=0, ignore_index=False)

#### Option 2: importing csv's with (relatively) clean text, preprocessing without tokenization with NLTK (= quicker!)

Default is one or more csv files with the filename ending in a year (for example text_1978.csv, 1851.csv, etc.). Make sure that the separator (sep=) is correctly defined (same as above) and that the text column has the correct header (from cols_to_keep)

In [ ]:
data = []  # Use a list to collect DataFrames

for file in tqdm(allfiles):
    # Load the file
    df = pd.read_csv(file, sep="\t", usecols=cols_to_keep, index_col=index_col)
    df['text'] = df['text'].str.lower()  # Lowercase text

    # Original word count per row
    df['original_word_count'] = df['text'].str.split().str.len()
    
    # Clean text
    df['text'] = df['text'].str.split()
    df['text_clean'] = [[w for w in text if w.isalpha() 
                         and len(w) >= 4] for text in df['text']]
    
    # Cleaned word count per row
    df['cleaned_word_count'] = df['text_clean'].str.len()
    
    # Words removed per row
    df['words_removed'] = df['original_word_count'] - df['cleaned_word_count']
    
    # Convert cleaned text back to a string for further processing if needed
    df['text_clean_str'] = df['text_clean'].apply(' '.join)

    # Append the processed DataFrame
    data.append(df)

# Concatenate all DataFrames at once
data = pd.concat(data, axis=0, ignore_index=False)

### Step 5: Save data to outdir

In [ ]:
def save_dataset(dataset):
    dataset_out = dataset.replace(" ", "_").lower()
    return dataset_out

In [ ]:
# Optional: remove columns that you do not need (to make the file saved below smaller)

data = data.drop(columns=[]) # add one or more columns like ['column_name_1', 'column_name_2']

In [ ]:
data.to_csv(outdir + '%s_clean.csv' %(save_dataset(dataset)), sep ='\t')

### (Optional) Step 6: Save dataframe with time index to outdir

These steps are only relevant if the index of 'data' above represents time. The exact formulation of 'format' below is contingent on the format of that time data (years, months and years, full dates, etc.). The code groups multiple index values with the same year together, so that the grouped dataframe can be saved as one big csv or, alternatively, as single csv's per year. It can even be saved as txt-files per year.

In [ ]:
data.index = pd.to_datetime(data.index, format ="%Y-%m-%d") # format depends on format in index (date) column. See https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
data = data.sort_index()input_as_string = " ".join(data['text'].tolist())input_as_string = " ".join(data['text'].tolist())

#### Option 1/3: Save df1 (grouped by year) as a single csv ####

In [ ]:
df1 = data.groupby(data.index.year).sum(numeric_only=False)

In [ ]:
df1.to_csv(outdir + '%s_year.csv' %(save_dataset(dataset)), sep='\t')

#### Option 2/3: Save as csv files per year ####

In [ ]:
sum_rows = 0

outdir_csv = outdir + '/%s_csv_per_year/' %(save_dataset(dataset))
os.makedirs(os.path.dirname(outdir_csv), exist_ok=True) # makes outdir if it doesn't exist already

for year in range(1900,2000, 1): # make sure to fill in the correct range in years!
    if len(data.loc[data.index.year == year]) > 0:
        df2 = data.loc[data.index.year == year]
        print(len(df2.index))
        sum_rows += len(df2.index)
        df2.to_csv(outdir_csv + str(year) + '.csv', sep='\t')
print("The number of rows in the original csv is " + str(len(data.index)))
print("The total number of rows of all new csvs is " + str(sum_rows))

#### Option 3/3: Save as txt files per year ####

In [ ]:
outdir_txt = outdir + '/%s_txt_per_year/' %(save_dataset(dataset))
os.makedirs(os.path.dirname(outdir_txt), exist_ok=True) # makes outdir if it doesn't exist already

for year in range(1900, 2000, 1): # make sure to fill in the correct range in years!
    if len(data.loc[data.index.year == year]) > 0:
        for index, row in data.iterrows():
            if index.year == year:
                with open(outdir_txt + str(year) + '.txt', 'a') as f:
                    f.write(str(row["text_clean_str"]))

### Step 7: Count total number of words

#### Step 7a: Visualize total number of words before and after preprocessing in a bar chart

In [ ]:
# Define the bar width and indices for rows
bar_width = 0.4
index = range(len(data))  # Use the row indices as x-axis

# Values for the bars
token_counts_before = data['original_word_count'].tolist()
token_counts_after = data['cleaned_word_count'].tolist()

# Create the dual bar chart
plt.figure(figsize=(12, 6))
plt.bar(index, token_counts_before, bar_width, label='Before Cleaning', alpha=0.7)
plt.bar([i + bar_width for i in index], token_counts_after, bar_width, label='After Cleaning', alpha=0.7)

# Add labels and title
plt.xlabel("Row Index")
plt.ylabel("Word Count")
plt.title("Word Counts Before and After Cleaning Per Row")
plt.xticks([i + bar_width / 2 for i in index], data.index.tolist(), rotation=90)  # Use the row indices as labels
plt.legend()
plt.tight_layout()
plt.show()

#### Step 7b: Count total number of words in your dataset before preprocessing

In [ ]:
print("The total number of words in \'%s\' before preprocessing is: %s" 
      %(str(indir), data['original_word_count'].sum()))

#### Step 7c: Count total number of words in your dataset after preprocessing

In [ ]:
print("The total number of words in \'%s\' after preprocessing is: %s" 
      %(str(indir), data['cleaned_word_count'].sum()))

#### Step 7d: Calculate total number of words removed by preprocessing

In [ ]:
print("The total number of tokens removed by preprocessing is: %s" 
      %(data['words_removed'].sum())